In [1]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import tensorflow.keras.applications.efficientnet as efn
import tensorflow as tf

In [2]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Flatten, Dense, Dropout
from tensorflow.keras.layers import DepthwiseConv2D
from tensorflow.keras.layers import Add
from tensorflow.keras.layers import Input, Conv2D, BatchNormalization, ReLU, Reshape, GlobalAveragePooling2D, Activation,UpSampling2D, AveragePooling2D

In [3]:
train_path = 'C:/Users/P M Shalini/Stanford/TRAIN_GRAY_SCALE'
valid_path = 'C:/Users/P M Shalini/Stanford/VALID_GRAY_SCALE'
test_path = 'C:/Users/P M Shalini/Stanford/TEST_GRAY_SCALE'

train_batches = ImageDataGenerator(preprocessing_function=efn.preprocess_input).flow_from_directory(
    directory=train_path, target_size=(224,224), batch_size=32)

Found 55220 images belonging to 49 classes.


In [4]:
valid_batches = ImageDataGenerator(preprocessing_function=efn.preprocess_input).flow_from_directory(
    directory=valid_path, target_size=(224,224), batch_size=32)

Found 1728 images belonging to 49 classes.


In [5]:
test_batches = ImageDataGenerator(preprocessing_function=efn.preprocess_input).flow_from_directory(
    directory=test_path, target_size=(224,224), batch_size=32, shuffle=False)

Found 1558 images belonging to 49 classes.


In [6]:
import tensorflow as tf
efficient = tf.keras.applications.EfficientNetB0()
efficient.summary()

Model: "efficientnetb0"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 rescaling (Rescaling)          (None, 224, 224, 3)  0           ['input_1[0][0]']                
                                                                                                  
 normalization (Normalization)  (None, 224, 224, 3)  7           ['rescaling[0][0]']              
                                                                                                  
 rescaling_1 (Rescaling)        (None, 224, 224, 3)  0           ['normalization[0][0

In [7]:
x = efficient.layers[-9].output

x = tf.keras.layers.Conv2D(filters=420, kernel_size=(1, 1))(x)
x = tf.keras.layers.BatchNormalization()(x)

x = tf.keras.layers.Conv2D(filters=1680, kernel_size=(1, 1))(x)
x = tf.keras.layers.BatchNormalization()(x)
x = tf.keras.layers.Activation('swish')(x)
x = tf.keras.layers.GlobalAveragePooling2D()(x)
x = tf.keras.layers.Dropout(0.1)(x)
x = tf.keras.layers.Dense(512, activation='relu')(x)
output = tf.keras.layers.Dense(49, activation='softmax')(x)

model = tf.keras.Model(inputs=efficient.input, outputs=output)

In [8]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 rescaling (Rescaling)          (None, 224, 224, 3)  0           ['input_1[0][0]']                
                                                                                                  
 normalization (Normalization)  (None, 224, 224, 3)  7           ['rescaling[0][0]']              
                                                                                                  
 rescaling_1 (Rescaling)        (None, 224, 224, 3)  0           ['normalization[0][0]']      

In [9]:
from tensorflow.keras.optimizers import Adam

optimizer = Adam(learning_rate=0.0001)
model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])

In [10]:
model.fit(x=train_batches,
            steps_per_epoch=len(train_batches),
            validation_data=valid_batches,
            validation_steps=len(valid_batches),
            epochs=5,
            verbose=1
)

Epoch 1/5
1726/1726 [==============================] - 10235s 6s/step - loss: 1.1837 - accuracy: 0.6663 - val_loss: 0.6718 - val_accuracy: 0.7934
Epoch 2/5
1726/1726 [==============================] - 26906s 16s/step - loss: 0.4191 - accuracy: 0.8694 - val_loss: 0.6790 - val_accuracy: 0.8090
Epoch 3/5
1726/1726 [==============================] - 10378s 6s/step - loss: 0.1955 - accuracy: 0.9386 - val_loss: 0.8409 - val_accuracy: 0.8003
Epoch 4/5
1726/1726 [==============================] - 10895s 6s/step - loss: 0.1287 - accuracy: 0.9593 - val_loss: 0.9614 - val_accuracy: 0.7934
Epoch 5/5
   1/1726 [..............................] - ETA: 2:51:42 - loss: 0.0236 - accuracy: 1.0000

KeyboardInterrupt: 

In [11]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix
import itertools

In [12]:
test_labels = test_batches.classes

In [13]:
predictions = model.predict(x=test_batches)
predicted_labels = np.argmax(predictions, axis=1)

# Calculate the testing accuracy
testing_accuracy = np.mean(predicted_labels == test_labels)

print("Testing Accuracy:", testing_accuracy)

49/49 [==============================] - 66s 1s/step
Testing Accuracy: 0.7997432605905006


In [14]:
model.save_weights('EfficientNetLite0_stand_modi_gray.h5')

In [15]:
train_path = 'C:/Users/P M Shalini/Stanford/TRAIN/'
valid_path = 'C:/Users/P M Shalini/Stanford/VALID/'

train_batches = ImageDataGenerator(preprocessing_function=efn.preprocess_input).flow_from_directory(
    directory=train_path, target_size=(224,224), batch_size=32)

Found 55220 images belonging to 49 classes.


In [16]:
valid_batches = ImageDataGenerator(preprocessing_function=efn.preprocess_input).flow_from_directory(
    directory=valid_path, target_size=(224, 224), batch_size=32)

Found 1728 images belonging to 49 classes.


In [17]:
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001),
              loss=tf.keras.losses.CategoricalCrossentropy(),
              metrics=['accuracy'])

In [18]:
model.fit(x=train_batches,
            steps_per_epoch=len(train_batches),
            validation_data=valid_batches,
            validation_steps=len(valid_batches),
            epochs=3,
            verbose=1
)

Epoch 1/3
1726/1726 [==============================] - 10766s 6s/step - loss: 0.1162 - accuracy: 0.9613 - val_loss: 0.9790 - val_accuracy: 0.7963
Epoch 2/3
 234/1726 [===>..........................] - ETA: 3:35:17 - loss: 0.0744 - accuracy: 0.9754

KeyboardInterrupt: 

In [19]:
test_path = 'C:/Users/P M Shalini/Stanford/TEST/'

In [20]:
test_batches = ImageDataGenerator(preprocessing_function=efn.preprocess_input).flow_from_directory(
    directory=test_path, target_size=(224,224), batch_size=32, shuffle=False)

Found 1558 images belonging to 49 classes.


In [21]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix
import itertools

In [22]:
test_labels = test_batches.classes

In [23]:
predictions = model.predict(x=test_batches)
predicted_labels = np.argmax(predictions, axis=1)

# Calculate the testing accuracy
testing_accuracy = np.mean(predicted_labels == test_labels)

print("Testing Accuracy:", testing_accuracy)

49/49 [==============================] - 106s 2s/step
Testing Accuracy: 0.8241335044929397


In [24]:
model.save_weights('EfficientNetLite0_stand_modi_rgb.h5')

In [25]:
def get_file_size(file_path):
    size = os.path.getsize(file_path)
    return size

In [26]:
def convert_bytes(size, unit=None):
    if unit == "KB":
        return print('File size: ' + str(round(size / 1024, 3)) + ' Kilobytes')
    elif unit == "MB":
        return print('File size: ' + str(round(size / (1024 * 1024), 3)) + ' Megabytes')
    else:
        return print('File size: ' + str(size) + ' bytes')

In [27]:
TF_LITE_MODEL_FILE_NAME = "EfficientNetLite0_stand_modi_rgb.tflite"

In [28]:
tf_lite_converter = tf.lite.TFLiteConverter.from_keras_model(model)
tf_lite_converter.optimizations = [tf.lite.Optimize.OPTIMIZE_FOR_SIZE]
# tf_lite_converter.optimizations = [tf.lite.Optimize.DEFAULT]
# tf_lite_converter.target_spec.supported_types = [tf.float16]
tflite_model = tf_lite_converter.convert()

INFO:tensorflow:Assets written to: C:\Users\PMSHAL~1\AppData\Local\Temp\tmphr58v0t3\assets


INFO:tensorflow:Assets written to: C:\Users\PMSHAL~1\AppData\Local\Temp\tmphr58v0t3\assets


In [29]:
tflite_model_name = TF_LITE_MODEL_FILE_NAME
open(tflite_model_name, "wb").write(tflite_model)

5844864

In [30]:
import os
convert_bytes(get_file_size(TF_LITE_MODEL_FILE_NAME), "MB")

File size: 5.574 Megabytes


In [ ]:
pwd

In [ ]:
import shutil

zip_file = "C:/Users/P M Shalini/Stanford/TEST_GRAY_SCALE.zip"
output_folder = "C:/Users/P M Shalini/Stanford/TEST_GRAY_SCALE"

shutil.unpack_archive(zip_file, output_folder)

In [ ]:
import shutil

directory_path = 'C:/Users/P M Shalini/Stanford/TRAIN'  # Replace with the actual path of the directory you want to delete

# Use try-except block to handle any exceptions that may occur during the deletion process
try:
    # Remove the directory recursively
    shutil.rmtree(directory_path)
    print(f"Directory '{directory_path}' deleted successfully.")
except FileNotFoundError:
    print(f"Directory '{directory_path}' not found.")
except PermissionError:
    print(f"Permission denied: Unable to delete directory '{directory_path}'.")
except Exception as e:
    print(f"An error occurred while deleting directory '{directory_path}': {str(e)}")